In [1]:
%pip install apify-client

   ---------------------------------------- 0.0/70.6 kB ? eta -:--:--
   ----------- ---------------------------- 20.5/70.6 kB 330.3 kB/s eta 0:00:01
   ----------------------------- ---------- 51.2/70.6 kB 525.1 kB/s eta 0:00:01
   ---------------------------------------- 70.6/70.6 kB 641.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os


# llm_config = {
#     'cache_seed': None,
#     'config_list': [
#         {
#             'model': 'bartowski/stable-code-instruct-3b-GGUF/stable-code-instruct-3b-Q3_K_S.gguf',
#             'base_url': "http://localhost:1234/v1",
#             "api_key": "lm-studio"
#         }
#     ]
# }

llm_config = {
    'config_list': [{
        'model': 'gpt-3.5-turbo',
        'OPENAI_API_KEY': os.getenv('OPENAI_API_KEY')}
    ]
}

import os
from dotenv import load_dotenv
load_dotenv() 


apify_api_key = os.getenv('APIFY_API_KEY')



SyntaxError: invalid syntax (1952262546.py, line 17)

In [11]:
from apify_client import ApifyClient
from typing_extensions import Annotated

def scrape_page(url: Annotated[str, 'The URL of the page to scrape']) -> Annotated[str, 'Content']:
    client = ApifyClient(token=apify_api_key)


    # Prepare the Actor input
    run_input = {
        "startUrls": [{"url": url}],
        "useSitemaps": False,
        "crawlerType": "playwright:firefox",
        "includeUrlGlobs": [],
        "excludeUrlGlobs": [],
        "ignoreCanonicalUrl": False,
        "maxCrawlDepth": 0,
        "maxCrawlPages": 1,
        "initialConcurrency": 0,
        "maxConcurrency": 200,
        "initialCookies": [],
        "proxyConfiguration": {"useApifyProxy": True},
        "maxSessionRotations": 10,
        "maxRequestRetries": 5,
        "requestTimeoutSecs": 60,
        "dynamicContentWaitSecs": 10,
        "maxScrollHeightPixels": 5000,
        "removeElementsCssSelector": """nav, footer, script, style, noscript, svg,
    [role=\"alert\"],
    [role=\"banner\"],
    [role=\"dialog\"],
    [role=\"alertdialog\"],
    [role=\"region\"][aria-label*=\"skip\" i],
    [aria-modal=\"true\"]""",
        "removeCookieWarnings": True,
        "clickElementsCssSelector": '[aria-expanded="false"]',
        "htmlTransformer": "readableText",
        "readableTextCharThreshold": 100,
        "aggressivePrune": False,
        "debugMode": True,
        "debugLog": True,
        "saveHtml": True,
        "saveMarkdown": True,
        "saveFiles": False,
        "saveScreenshots": False,
        "maxResults": 9999999,
        "clientSideMinChangePercentage": 15,
        "renderingTypeDetectionPercentage": 10,
    }

    # Run the Actor and wait for it to finish
    client.actors().create(name='scrape-llm-actor')
    run = client.actor("scrape-llm-actor").call(run_input=run_input)

    # Fetch and print Actor results from the run's dataset (if there are any)
    text_data = ""
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        text_data += item.get("text", "") + "\n"

    average_token = 0.75
    max_tokens = 20000  # slightly less than max to be safe 32k
    text_data = text_data[: int(average_token * max_tokens)]
    return text_data


In [12]:
from autogen import ConversableAgent, register_function

# Create web scrapper agent.
scraper_agent = ConversableAgent(
    "WebScraper",
    llm_config=llm_config,
    system_message="You are a web scrapper and you can scrape any web page using the tools provided. "
    "Returns 'TERMINATE' when the scraping is done.",
)

# Create user proxy agent.
user_proxy_agent = ConversableAgent(
    "UserProxy",
    llm_config=False,  # No LLM for this agent.
    human_input_mode="NEVER",
    code_execution_config=False,  # No code execution for this agent.
    is_termination_msg=lambda x: x.get("content", "") is not None and "terminate" in x["content"].lower(),
    default_auto_reply="Please continue if not finished, otherwise return 'TERMINATE'.",
)

# Register the function with the agents.
register_function(
    scrape_page,
    caller=scraper_agent,
    executor=user_proxy_agent,
    name="scrape_page",
    description="Scrape a web page and return the content.",
)




We used the `reflection_with_llm` option for summary method to perform the formatting of the output into a desired format. The summary method is called after the conversation is completed given the complete history of the conversation.

In [13]:
chat_result = user_proxy_agent.initiate_chat(
    scraper_agent,
    message="Can you scrape agentops.ai for me?",
    summary_method="reflection_with_llm",
    summary_args={
        "summary_prompt": """Summarize the scraped content and format summary EXACTLY as follows:
---
*Company name*:
`Acme Corp`
---
*Website*:
`acmecorp.com`
---
*Description*:
`Company that does things.`
---
*Tags*:
`Manufacturing. Retail. E-commerce.`
---
*Takeaways*:
`Provides shareholders with value by selling products.`
---
*Questions*:
`What products do they sell? How do they make money? What is their market share?`
---
"""
    },
)

UserProxy (to WebScraper):

Can you scrape agentops.ai for me?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Can you scrape agentops.ai for me?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
WebScraper (to UserProxy):

I'm an AI language model, so I don't have the ability to directly scrape content from websites. However, I can certainly guide you on how to do it using Python and libraries like `requests` and `BeautifulSoup`.

Here's a basic example of how you might set up your scraping script:

```python
import requests
from bs4 import BeautifulSoup

def scrape_agentops(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    return soup

url = "https://www.agentops.ai"

soup = scrape_agentops(url)
```

This is a basic example and might need to be adjusted based on the specific structure of the web page you're 